## import

In [1]:
import os
import random
import numpy as np
import torchaudio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from torch_audiomentations import Compose, Gain, PolarityInversion
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt
import torchaudio.transforms as T

## Distribution of raw data length

In [ ]:
dataset_dir = r"C:\Users\omen\Desktop\LibriSpeech" 
sample_rate=16000

X = []
y = []
labels = sorted(os.listdir(dataset_dir))
sample_counts=[]
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_dir):
        continue
    
    files = [os.path.join(root, file)
                 for root, _, file_list in os.walk(label_dir)
                 for file in file_list
                 if file.endswith('.wav') or file.endswith('.flac')]
    for file in files:
        audio, sr = torchaudio.load(file)
        if sr != sample_rate:
            audio = torchaudio.functional.resample(audio, sr, sample_rate)
        print(audio.shape)      
        num_samples = audio.shape[-1]
        sample_counts.append(num_samples)
median_samples = np.min(sample_counts) if sample_counts else 0          
print(f"The median number of samples in the dataset is: {median_samples}")
print("total sample size：",len(sample_counts))

# Data slicing, saving

In [ ]:
new_data_dir=r"C:\Users\omen\Desktop\train-clean-360" 


def create_output_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)


def save_audio_slice(audio, label, file_name, index):
    file_dir = os.path.join(new_data_dir, label)
    create_output_dir(file_dir)
    
    new_file_name = file_name.replace('.flac', f'-{index}.flac').replace('.wav', f'-{index}.wav')
    new_file_path = os.path.join(file_dir, new_file_name)
    torchaudio.save(new_file_path, audio, sample_rate)
    print(f"Saved slice: {new_file_path}")

## The data is sliced into 10,000 pieces and saved.

In [ ]:
dataset_dir = r"C:\Users\omen\Desktop\train-clean-360"
sample_rate=16000

X = []
y = []
labels = sorted(os.listdir(dataset_dir))
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_dir):
        continue
    
    files = [os.path.join(root, file)
                 for root, _, file_list in os.walk(label_dir)
                 for file in file_list
                 if file.endswith('.wav') or file.endswith('.flac')]
    for file in files:
        audio, sr = torchaudio.load(file)
        if sr != sample_rate:
            audio = torchaudio.functional.resample(audio, sr, sample_rate)
    

       
        relative_path = os.path.relpath(file, dataset_dir)
        label_path = os.path.dirname(relative_path)  
        file_name = os.path.basename(file)

        audio_length=audio.shape[1]
        if audio_length<20000:
            audio=audio[:,:10000]   #If the length is less than 2w, truncate to 1w.
            save_audio_slice(audio,label_path,file_name,1)
        else:
            num_slices=audio_length//10000
            for i in range(num_slices):
                start=i*10000
                end=(i+1)*10000
                audio_slice=audio[:,start:end]
                save_audio_slice(audio_slice,label_path,file_name,i+1)


## Data length after slicing

In [ ]:
dataset_dir = r"C:\Users\omen\Desktop\dev-other-cut" 
sample_rate=16000

X = []
y = []
labels = sorted(os.listdir(dataset_dir))
sample_counts=[]
for label in labels:
    label_dir = os.path.join(dataset_dir, label)
    if not os.path.isdir(label_dir):
        continue
    
    files = [os.path.join(root, file)
                 for root, _, file_list in os.walk(label_dir)
                 for file in file_list
                 if file.endswith('.wav') or file.endswith('.flac')]
    for file in files:
        audio, sr = torchaudio.load(file)
        if sr != sample_rate:
            audio = torchaudio.functional.resample(audio, sr, sample_rate)
        print(audio.shape)      
        num_samples = audio.shape[-1]
        sample_counts.append(num_samples)
median_samples = np.min(sample_counts) if sample_counts else 0      
print(f"The median number of samples in the dataset is: {median_samples}")
print("total sample size：",len(sample_counts))